In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import os
import groupby_toolz
import spacy #https://spacy.io/usage/linguistic-features
# install spacy here https://spacy.io/usage
# import en_core_web_sm
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from IPython.display import display
pd.options.display.max_columns = None
from decimal import *
TWOPLACES = Decimal(10) ** -2
# https://stackoverflow.com/questions/21903842/how-do-i-compare-two-strings-in-python

In [2]:
# Finds the first digit (basically skips over words and letters to find the first number)
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

# i finds the first digit: if it equals zero then there is not a digit, if it equals the end then leave it
def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

            
# NATURAL SORT
######################################################################
# sort the number naturally so 5 will come before 15
def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))
# x['matches']= x['matches'].apply(lambda x: sorted(x, key=natural_key))



#------------atof and natural_keys are complementary-------------
# atof and natural_keys: Will naturally sort 5 infront of 15
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
#########################################################################





# REPLACE
#########################################################################
# # Replace specific text in a string: Exmapel1  to  Example1
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# replacement={'Exmapel1':'Example1',    }
# df= df.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)


def replace_all(text, dic):
    '''
    Given a text and replacement dictionary, performs multiple str.replace()
    Arguments:
    ==========
    text: str, text to perform str.replace()
    dic: dict, where each value is the string to replace, and each key is the string to replace with
    '''
    for key, value in dic.items():
        for v in value:
            text = text.replace(v, key)
    return text
# power['min_temp_sj']=power['min_temp_sj'].apply(lambda x: replace_all(x,{'':['Ã‚Â°F']}))


# Replace all letters and some special characters with a blank
# data['max']=data['max'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
#########################################################################




# FRACTIONS
#########################################################################
# Convert a fraction that does not have an int in front of the fraction "1/3"
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))
#########################################################################





# CONVERTING FRACTIONS, WHOLE NUMBERS, AND NUMBERS WITH LETTERS
#########################################################################
#Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))



 
# converts the value to a float if possible and skips the value if not   
def convert_float(val):
    try:
        return float(val)
    except:
        return str(val)
    
    
    

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x
 # Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  



def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
    #         print(numbers)
    #         numbers = literal_eval(numbers)
    #         print(numbers)
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
#         print(numbers)
        return numbers



# Just merging everything into one function
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: round_string_float(x))     
# final_done(power, 'height_sj_one')
#########################################################################


# TEMP RANGES
#########################################################################
# Converts Above 40 Degrees F  and Above 0
def above_freezing(numbers): 
    if numbers.find('above freezing') > -1:   
        numbers=0
        return numbers
    if numbers.find('above ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    else:
        return numbers
# data['min']=data['min'].apply(lambda x: above_freezing(x))
    
#Splits temp ranges: 60 to 100 Degrees F, -30 to 200 Degrees F, 50-100 Degrees
def split_temp(data, col, dimension):
#     data['matches']=data['matches'].str.get(0) # Use this to for arrays
    data[col]=data[col].apply(lambda x:x.lower()) #sometimes x or to is capitalized like X or To and it wont split
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split(' to |(?<=\d)-',expand=True,) # have to split on "to" as well as ranges "50-100" but have to keep "-30" from "-30 to 200" 
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
# split_temp(power, 'operating_temp_sj')    

In [3]:
# path with import 
# Need File stream setup to access this from local machine
f_path=r'G:\Shared drives\GroupBy Public\Customer Success\.Enrich\ScrapedData\Motion - Syndication Deliverable\Final Delivery\_previously_sent\delivered_unified'
csvs = sorted([file for file in os.listdir(f_path) if file[-len('csv'):] == 'csv'])

In [4]:
# Drop zero vector columns
df = pd.read_csv(f'{f_path}/{csvs[6]}', dtype=str)
print(len(df.columns.tolist())) #print number of columns before dropping zeros
zero_col=pd.DataFrame(df.notnull().sum())
zero_col=zero_col.reset_index()
zero_col.columns = ['col', 'not_null']
df.drop(zero_col[zero_col['not_null'].astype(str)<'1']['col'], axis =1, inplace = True)
print(len(df.columns.tolist())) # print number of relavent columns
new_col=zero_col[zero_col['not_null'].astype(str)>'1'][3:].reset_index(drop=True) # first 11 columns are the url/external_id/etc.. and we will not drop these
first=new_col['col'][0]
second=new_col['col']

697
659


In [5]:
power=df

In [6]:
power['od_sj']=power[[
'Outside_Diameter_Inch',
'OD',
'Outside_Dia_In',
]].apply(lambda x: ",".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
power['od_sj'].explode().value_counts()

            3125
2.667 in       1
3.25           1
13/16 in       1
1 in           1
2.165          1
Name: od_sj, dtype: int64

In [7]:
final_done(power,'od_sj').explode().value_counts()

        3125
1          1
3.25       1
0.81       1
2.16       1
2.67       1
Name: od_sj, dtype: int64

In [8]:
power['depth_sj']=power[[
'Depth_In',
'Depth',
]].apply(lambda x: ",".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
power['depth_sj'].explode().value_counts()

           3109
6"            3
0.65"         2
1-1/2"        2
23-3/4        2
28-3/4        2
4"            2
58-1/2        1
23-3/4"       1
1.25"         1
0.77"         1
31-3/4        1
1.658"        1
7             1
0.88"         1
Name: depth_sj, dtype: int64

In [9]:
final_done(power,'depth_sj').explode().value_counts()

         3109
6           3
23.75       3
4           2
28.75       2
1.5         2
0.65        2
1.25        1
7           1
31.75       1
0.77        1
58.5        1
0.88        1
1.66        1
Name: depth_sj, dtype: int64

In [10]:
power['height_sj_one']=power[[
    'Height',
    'Height_In',
]].apply(lambda x: ",".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
power['height_sj_one'].explode().value_counts()

              3082
67-7/8"          4
1.35"            3
36-1/4"          2
1.83"            2
65 in            2
1.88"            2
2.32"            2
1.92"            2
5 21/32 in       1
13 in            1
25"              1
36"              1
14 in            1
63-1/2           1
10-3/4           1
24"              1
44"              1
60"              1
1.01"            1
3"               1
1.13"            1
1/2"             1
4-15/16"         1
17 3/4 in        1
8 1/4 in         1
2-3/8"           1
4.45"            1
1.08"            1
49 1/2 in        1
12-1/2           1
1.70"            1
2.28 in          1
1.12"            1
0.91"            1
41 in            1
1.62"            1
2"               1
Name: height_sj_one, dtype: int64

In [11]:
final_done(power,'height_sj_one').explode().value_counts()
# power['height_sj_two']=power['height_sj_one'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: round_string_float(x))
# power[['height_sj_one','height_sj_two']][2500:3000]

         3082
67.88       4
1.35        3
36.25       2
1.92        2
1.88        2
1.83        2
2.32        2
65          2
1.01        1
13          1
4.45        1
1.7         1
4.94        1
3           1
8.25        1
5.66        1
0.91        1
10.75       1
1.13        1
1.62        1
1.12        1
12.5        1
1.08        1
0.5         1
2.28        1
14          1
17.75       1
2           1
63.5        1
60          1
49.5        1
44          1
41          1
36          1
25          1
24          1
2.38        1
Name: height_sj_one, dtype: int64

In [12]:
power['operating_temp_sj']=power[[
'Application_Temperature',
'Operating_Temp',
'Operating_Temp_Range',
]].apply(lambda x: ",".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
power['operating_temp_sj'].explode().value_counts()

                                 2861
50 Degrees  to 100 Degrees F       45
50 Degrees  to 85 Degrees F        35
50 Degrees  to 90 Degrees F        30
50 to 90 Degrees F                 22
50 to 100 Degrees F                16
40 Degrees  to 90 Degrees F        12
32 to 100 Degrees F                11
40 Degrees  to 100 Degrees F       10
50 to 110 Degrees F                 6
50 to 85 Degrees F                  6
60 Degrees  to 100 Degrees F        5
60 to 100 Degrees F                 5
32 Degrees  to 100 Degrees F        4
-30 to 200 Degrees F                4
60 to 90 Degrees F                  3
32 Degrees  to 122 Degrees F        3
Above 40 Degrees F                  3
40 to 90 Degrees F                  3
32 to 122 Degrees F                 3
0 Degrees  to 100 Degrees F         2
40 Degrees  to 120 Degrees F        2
50 to 95 Degrees F                  2
14 Degrees  to 122 Degrees F        2
Above Freezing                      2
60 to 80 Degrees F                  2
50 Degrees  

In [13]:
split_temp(power, 'operating_temp_sj','temp')
power['min_temp'].explode().value_counts()

       2863
50      169
40       32
32       25
60       16
0         7
-30       4
14        3
70        2
36        1
49        1
45        1
65        1
55        1
18        1
48        1
4         1
30        1
Name: min_temp, dtype: int64